Python wrapper for the JTAG Atlantic communiation library

Adapted from http://alterawiki.com/wiki/High_Speed_Image_Download_Demo

In [ ]:
import ctypes
import numpy as np

In [ ]:
# requiring jtag_atlantic.dll and jtag_client.dll put in the same folder as this notebook
dll_path = 'jtag_atlantic.dll'

In [ ]:
atlantic_dll = ctypes.cdll.LoadLibrary(dll_path)

# Function definitions

In [ ]:
"""
JTAGATLANTIC * jtagatlantic_open (
    const char * chain,
    int device_index,
    int link_instance,
    const char * app_name);
"""
JAOpen = getattr(atlantic_dll,'?jtagatlantic_open@@YAPEAUJTAGATLANTIC@@PEBDHH0@Z')  # mangled name obtained from dllwalker
JAOpen.argtypes = [ctypes.c_char_p, ctypes.c_int64, ctypes.c_int64, ctypes.c_char_p]
JAOpen.restype = ctypes.c_void_p

"""
int  jtagatlantic_get_error (const char * * other_info);
"""
JAGetError = getattr(atlantic_dll,'?jtagatlantic_get_error@@YA?AW4JATL_ERROR@@PEAPEBD@Z')
JAGetError.argtypes = [ctypes.c_void_p]
JAGetError.restype = ctypes.c_int64

"""
void jtagatlantic_close (JTAGATLANTIC * link);
"""
JAClose = getattr(atlantic_dll,'?jtagatlantic_close@@YAXPEAUJTAGATLANTIC@@@Z')
JAClose.argtypes = [ctypes.c_void_p]
JAClose.restype = None

"""
int  jtagatlantic_write (JTAGATLANTIC * link, const char * data, unsigned int count);
"""
JAWrite = getattr(atlantic_dll,'?jtagatlantic_write@@YAHPEAUJTAGATLANTIC@@PEBDI@Z')
JAWrite.argtypes = [ctypes.c_void_p, ctypes.c_char_p, ctypes.c_uint64]
JAWrite.restype = ctypes.c_int64

"""
int  jtagatlantic_flush (JTAGATLANTIC * link);
"""
JAFlush = getattr(atlantic_dll,'?jtagatlantic_flush@@YAHPEAUJTAGATLANTIC@@@Z')
JAFlush.argtypes = [ctypes.c_void_p]
JAFlush.restype = ctypes.c_int64

"""
int  jtagatlantic_read (JTAGATLANTIC * link, char * buffer, unsigned int buffsize);
"""
JARead = getattr(atlantic_dll,'?jtagatlantic_read@@YAHPEAUJTAGATLANTIC@@PEADI@Z')
JARead.argtypes = [ctypes.c_void_p, ctypes.c_char_p, ctypes.c_uint64]
JARead.restype = ctypes.c_int64

In [ ]:
# information obtained from running nios2-terminal.exe
cable_name = b"DE-SoC [USB-1]"  # related to the board you are using
device_index = 2  # related to the JTAG device on the board. 0: to let the DLL determine itself. 
                  #      1: usually the on-board FPGA "burner" chip, 2: the FPGA chip
link_instance = 0 # -1: to let the DLL determine the JTAG instance in the chip. 0: in my case I have only 1 instance
                  #      of JTAG-UART so it is indexed by 0.

# establish the link
link = JAOpen(
    ctypes.c_char_p(cable_name),
    ctypes.c_int64(device_index),
    ctypes.c_int64(link_instance),
    ctypes.c_char_p(b'')
)

# Load test image

In [ ]:
%pylab inline
import skimage.io as skio
import skimage.transform as sktr

In [ ]:
img = skio.imread('./CAL_PRI_LOGO_ON_NAV_launch.png')
img = img[:,:,:3].sum(axis=2)
img = sktr.resize(img, (1024,1024))
img = (img != 0).astype(np.uint8)*255
img[-1,-1] = 255
img.dtype

In [ ]:
imshow(img.astype(np.float64))
colorbar()

In [ ]:
img2 = skio.imread('image347.jpg')
img2.shape, img2.dtype

In [ ]:
177-134

In [ ]:
img2 = img2[:-43,:,:].astype(float64).sum(axis=2)
img2 = sktr.resize(img2,(1024,1024), mode='constant')
img2 = (img2/img2.max()*255).astype(np.uint8)
img2[img2==254] = 255

# Send (cmd 0b10??????)
To do: write a function for it and take care of value 254 to be repeated.

In [ ]:
img_bytes = img.tobytes() + img2.tobytes()
len(img_bytes)

In [ ]:
num_images = 2
assert 1 <= num_images and num_images <= 64
cmd = bytes([0x80+num_images-1])
to_send = cmd + img_bytes + b'\xFE\x01'
print('to send',len(to_send),'bytes')
JAFlush(link)
for i in range(0, len(to_send), 10000):
    to_send_chunk = to_send[i:i+10000]
    JAWrite(link, ctypes.c_char_p(to_send_chunk), len(to_send_chunk))
    JAFlush(link)

# Send Single (cmd 0b01??????)
To do: write a function for it and take care of value 254 to be repeated.

In [ ]:
img_bytes = img.tobytes()
len(img_bytes)

In [ ]:
frame_id = 2
assert 0 <= frame_id and frame_id <= 63
cmd = bytes([0x40+frame_id])
to_send = cmd + img_bytes + b'\xFE\x01'
print('to send',len(to_send),'bytes')
JAFlush(link)
for i in range(0, len(to_send), 10000):
    to_send_chunk = to_send[i:i+10000]
    JAWrite(link, ctypes.c_char_p(to_send_chunk), len(to_send_chunk))
    JAFlush(link)

# Total Number of Frames (cmd 0x07)

In [ ]:
def sendNumFrames(link, num_frames):
    if num_frames <= 0 or num_frames > 64:
        raise ValueError('The value of number of frames can only be within 1 to 64.')
    num_frames = np.uint8(num_frames)
    cmd = b'\x07'
    JAFlush(link)
    to_send = cmd + bytes([num_frames]) + b'\xFE\x01'
    n_bytes_sent = JAWrite(link, ctypes.c_char_p(to_send), len(to_send))
    JAFlush(link)
    return (n_bytes_sent==len(to_send))

In [ ]:
sendNumFrames(link, 3)

# Offsets (cmd 0x01, 0x02)

In [ ]:
def sendOffsetX(link, offset_x_value=0):
    if abs(offset_x_value) > 128:
        raise ValueError('The absolute value of offset should not exceed 128')
    cmd = b'\x01'
    JAFlush(link)
    to_send_offset_x = cmd + bytes([abs(offset_x_value), (offset_x_value<0)]) + b'\xFE\x01'
    n_bytes_sent = JAWrite(link, ctypes.c_char_p(to_send_offset_x), len(to_send_offset_x))
    JAFlush(link)
    return (n_bytes_sent==len(to_send_offset_x))

In [ ]:
def sendOffsetY(link, offset_y_value=0):
    if abs(offset_y_value) > 128:
        raise ValueError('The absolute value of offset should not exceed 128')
    cmd = b'\x02'
    JAFlush(link)
    to_send_offset_y = cmd + bytes([abs(offset_y_value), (offset_y_value<0)]) + b'\xFE\x01'
    n_bytes_sent = JAWrite(link, ctypes.c_char_p(to_send_offset_y), len(to_send_offset_y))
    JAFlush(link)
    return (n_bytes_sent==len(to_send_offset_y))

In [ ]:
sendOffsetY(link,128)

# Cycles of Display (0x03)

In [ ]:
def sendCyclesOfDisplay(link, n_cycles=1):
    if abs(n_cycles) >= (1<<16):
        raise ValueError('The value should not exceed 65535')
    n_cycles = np.uint16(n_cycles)
    n_cycles_u = np.uint8((n_cycles & 0xFF00) >> 8)
    n_cycles_l = np.uint8((n_cycles & 0x00FF)     )
    cmd = b'\x03'
    JAFlush(link)
    to_send = cmd + bytes([n_cycles_l, n_cycles_u]) + b'\xFE\x01'
    n_bytes_sent = JAWrite(link, ctypes.c_char_p(to_send), len(to_send))
    JAFlush(link)
    return (n_bytes_sent==len(to_send))

In [ ]:
sendCyclesOfDisplay(link,200)

# Trigger the sequencing (0x04)

In [ ]:
def triggerSequencing(link):
    cmd = b'\x04'
    JAFlush(link)
    to_send = cmd + b'\xFE\x01'
    n_bytes_sent = JAWrite(link, ctypes.c_char_p(to_send), len(to_send))
    JAFlush(link)
    return (n_bytes_sent==len(to_send))

In [ ]:
triggerSequencing(link)

# Galvo values (0x05, 0x06)

In [ ]:
def _sendGalvoValues(link, value, cmd):
    if value >= (1 << 24):
        raise ValueError('The value should not exceed %d' % (1<<24-1))
    value = np.uint32(value)
    value_0 = np.uint8((value & 0x0000FF)     )
    value_1 = np.uint8((value & 0x00FF00) >> 8)
    value_2 = np.uint8((value & 0xFF0000) >> 16)
    JAFlush(link)
    to_send = cmd + bytes([value_0,value_1,value_2]) + b'\xFE\x01'
    n_bytes_sent = JAWrite(link, ctypes.c_char_p(to_send), len(to_send))
    JAFlush(link)
    return (n_bytes_sent==len(to_send))

def sendGalvoValuesX(link, value):
    return _sendGalvoValues(link, value, b'\x05')
def sendGalvoValuesY(link, value):
    return _sendGalvoValues(link, value, b'\x06')

In [ ]:
sendGalvoValuesX(link, 14223)

# Close connection

In [ ]:
JAClose(link)